In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import sklearn
import torchvision
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import seaborn as sns
import glob
from pathlib import Path
torch.manual_seed(1)
np.random.seed(1)

In [2]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [10]:
from pathlib import Path
cwd = Path().resolve()
root = cwd.parent
data_path = root / 'data'

In [11]:
root

PosixPath('/Users/gregruyoga/gmoneycodes/retina')

In [12]:
train_files = []
valid_files = []
test_files = []

for file in os.listdir(data_path / 'Training_Set' / 'Training_Set' / 'Training'):
    train_files.append(file)

for file in os.listdir(data_path / 'Evaluation_Set' / 'Evaluation_Set' / 'Validation'):
    valid_files.append(file)

for file in os.listdir(data_path / 'Test_Set' / 'Test_Set' / 'Test'):
    valid_files.append(file)

In [15]:
train_labels = pd.read_csv(data_path / 'Training_Set' / 'Training_Set' /  'RFMiD_Training_Labels.csv')

valid_labels = pd.read_csv(data_path / 'Evaluation_Set' / 'Evaluation_Set' /  'RFMiD_Validation_Labels.csv')

test_labels = pd.read_csv(data_path / 'Test_Set' / 'Test_Set' / 'RFMiD_Testing_Labels.csv')

train_ids = []
for element in train_files:
    train_ids.append(element.split('.')[0])

valid_ids = []
for element in valid_files:
    valid_ids.append(element.split('.')[0])

test_ids = []
for element in test_files:
    test_ids.append(element.split('.')[0])

train_ids = pd.Series(train_ids, name='ids')
train_files = pd.Series(train_files, name='filenames')
train_files = pd.concat([train_ids, train_files], axis=1)

valid_ids = pd.Series(valid_ids, name='ids')
valid_files = pd.Series(valid_files, name='filenames')
valid_files = pd.concat([valid_ids, valid_files], axis=1)

test_ids = pd.Series(test_ids, name='ids')
test_files = pd.Series(test_files, name='filenames')
test_files = pd.concat([test_ids, test_files], axis=1)

In [16]:
train_files['ids'] = train_files['ids'].astype('int64')
valid_files['ids'] = valid_files['ids'].astype('int64')
test_files['ids'] = test_files['ids'].astype('int64')

train_df = pd.merge(train_labels, train_files, left_on='ID', right_on='ids')
valid_df = pd.merge(valid_labels, valid_files, left_on='ID', right_on='ids')
test_df = pd.merge(test_labels, test_files, left_on='ID', right_on='ids')
train_df

,ID,Disease_Risk,DR,ARMD,MH,DN,MYA,BRVO,TSLN,ERM,...,CF,VH,MCA,VS,BRAO,PLQ,HPED,CL,ids,filenames
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1.png
1,2,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2.png
2,3,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,3.png
3,4,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,4.png
4,5,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,5.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,1916,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1916,1916.png
1916,1917,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1917,1917.png
1917,1918,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1918,1918.png
1918,1919,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1919,1919.png


In [19]:
train_df['full_file_paths'] = '../data/Training_Set/Training_Set/Training/' + train_df['filenames']
valid_df['full_file_paths'] = '../data/Evaluation_Set/Evaluation_Set/Validation/' + valid_df['filenames']
test_df['full_file_paths'] = '../data/Test_Set/Test_Set/Test/' + test_df['filenames']

In [18]:
class RetinalDataset(torch.utils.data.Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # last column = full_file_paths, as you created earlier
        img_path = self.df.iloc[idx, -1]

        # label columns: from 1 to -3 (same as before)
        label_array = self.df.iloc[idx, 1:-3].values.astype("float32")

        # read image
        image = Image.open(img_path).convert('RGB')
        image = np.array(image)

        # albumentations transform
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        else:
            image = transforms.ToTensor()(image)

        labels = torch.from_numpy(label_array)
        return image, labels

In [22]:
train_df

,ID,Disease_Risk,DR,ARMD,MH,DN,MYA,BRVO,TSLN,ERM,...,VH,MCA,VS,BRAO,PLQ,HPED,CL,ids,filenames,full_file_paths
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1.png,../data/Training_Set/Training_Set/Training/1.png
1,2,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,2.png,../data/Training_Set/Training_Set/Training/2.png
2,3,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,3.png,../data/Training_Set/Training_Set/Training/3.png
3,4,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,4,4.png,../data/Training_Set/Training_Set/Training/4.png
4,5,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,5,5.png,../data/Training_Set/Training_Set/Training/5.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,1916,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1916,1916.png,../data/Training_Set/Training_Set/Training/191...
1916,1917,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1917,1917.png,../data/Training_Set/Training_Set/Training/191...
1917,1918,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1918,1918.png,../data/Training_Set/Training_Set/Training/191...
1918,1919,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1919,1919.png,../data/Training_Set/Training_Set/Training/191...


In [23]:
train_transforms = A.Compose([
    A.Resize(1424, 2144),
    A.HorizontalFlip(),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

test_transforms = A.Compose([
    A.Resize(1424, 2144),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

In [25]:
train_dataset = RetinalDataset(df=train_df, transform=train_transforms)
valid_dataset = RetinalDataset(df=valid_df, transform=test_transforms)
test_dataset = RetinalDataset(df=test_df, transform=test_transforms)

In [26]:
batch_size = 64

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True
)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=10, shuffle=False, num_workers=2, pin_memory=True
)

In [27]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [28]:
model = torchvision.models.resnet50(pretrained=True)
model.fc = nn.Sequential(
               nn.Linear(2048, 46))

/opt/homebrew/Caskroom/miniconda/base/envs/retinann/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/retinann/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/gregruyoga/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100.0%


In [30]:
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, weight_decay=0.0005, lr=0.0001)

criterion = nn.BCEWithLogitsLoss(reduction='sum')

lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=8, cooldown=10)

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
from sklearn.metrics import f1_score
epochs = 100

total_train_loss = []
total_valid_loss = []
best_valid_loss = np.Inf

for epoch in range(epochs):
    print('Epoch: ', epoch + 1)
    train_loss = []
    valid_loss = []

    # -------- TRAIN --------
    model.train()
    for image, target in train_loader:
        image  = image.to(device)
        target = target.to(device).float()

        optimizer.zero_grad()
        output = model(image)
        loss   = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())

    # -------- VALID + F1 --------
    all_targets = []
    all_preds   = []

    model.eval()
    with torch.no_grad():
        for image, target in valid_loader:
            image  = image.to(device)
            target = target.to(device).float()

            output = model(image)
            loss   = criterion(output, target)
            valid_loss.append(loss.item())

            # probs & preds for F1
            probs = torch.sigmoid(output).cpu().numpy()      # [B, 46]
            preds = (probs > 0.5).astype(int)               # threshold 0.5

            targets_np = target.cpu().numpy()               # [B, 46]

            all_targets.append(targets_np)
            all_preds.append(preds)

    epoch_train_loss = np.mean(train_loss)
    epoch_valid_loss = np.mean(valid_loss)
    print(f'Epoch {epoch + 1}, train loss: {epoch_train_loss:.4f}, valid loss: {epoch_valid_loss:.4f}')

    # Save best model by valid loss
    if epoch_valid_loss < best_valid_loss:
        torch.save(model.state_dict(), 'retinal_disease.pt')
        print('Model improved. Saving model.')
        best_valid_loss = epoch_valid_loss

    lr_scheduler.step(epoch_valid_loss)
    total_train_loss.append(epoch_train_loss)
    total_valid_loss.append(epoch_valid_loss)

    # Stack and compute F1
    all_targets = np.vstack(all_targets)   # shape [N_val, 46]
    all_preds   = np.vstack(all_preds)

    f1_macro = f1_score(all_targets, all_preds, average='macro')
    f1_micro = f1_score(all_targets, all_preds, average='micro')
    print("Val F1 macro:", f1_macro, " | F1 micro:", f1_micro)